# Load packages

In [1]:
from google.cloud import bigquery
from google.oauth2 import service_account

ModuleNotFoundError: No module named 'google'

# Authenticate with Google BigQuery

In [ ]:


# TODO(developer): Set key_path to the path to the service account key
#                  file.
# key_path = "path/to/service_account.json"
key_path = ""
credentials = service_account.Credentials.from_service_account_file(
    key_path,
    scopes=["https://www.googleapis.com/auth/cloud-platform"],
)

client = bigquery.Client(
    credentials=credentials,
    project=credentials.project_id,
)

In [ ]:
# Load data into pandas dataframe

In [ ]:
# Exploratory data analysis 
* See Roger Pengs book

In [ ]:
# Gather 

In [ ]:
# Assess

In [ ]:
# Clean

In [ ]:
# Analyze

In [ ]:
# Model

In [ ]:
# Visualize